In [17]:
from sklearn.preprocessing import MinMaxScaler

from common.utils import load_data, mape, rmse, compar


vibration = load_data('/home/sachin/Thesis/data')[['vibration']]



train_start_dt = '2021-05-27 06:00:00'
test_start_dt = '2021-05-27 12:00:00'
test_end_dt = '2021-05-27 13:00:00'

scaler = MinMaxScaler()

result_dict = dict()
iteration_dict = dict()


train = vibration.copy()[(vibration.index >= train_start_dt) & (vibration.index < test_start_dt)][['vibration']]
test = vibration.copy()[(vibration.index >= test_start_dt) & (vibration.index < test_end_dt)][['vibration']]
print(train.values)

[[0.66]
 [0.66]
 [0.75]
 ...
 [0.84]
 [0.8 ]
 [0.79]]


In [18]:
print(test.values)

[[0.81]
 [0.74]
 [0.7 ]
 ...
 [0.56]
 [0.57]
 [0.57]]


In [19]:
print(train.values)

train['vibration'] = scaler.fit_transform(train)

test['vibration'] = scaler.transform(test)

train_data = train.values
test_data = test.values


[[0.66]
 [0.66]
 [0.75]
 ...
 [0.84]
 [0.8 ]
 [0.79]]


In [20]:
import numpy as np
from time import time
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

timesteps = 5

start = time()

train_data_timesteps = np.array(
    [[j for j in train_data[i:i + timesteps]] for i in range(0, len(train_data) - timesteps + 1)])[:, :, 0]
test_data_timesteps = np.array(
    [[j for j in test_data[i:i + timesteps]] for i in range(0, len(test_data) - timesteps + 1)])[:, :, 0]

x_train, y_train = train_data_timesteps[:, :timesteps - 1], train_data_timesteps[:, [timesteps - 1]]
x_test, y_test = test_data_timesteps[:, :timesteps - 1], test_data_timesteps[:, [timesteps - 1]]

print(y_test)

model = SVR(kernel='rbf', gamma=1, C=10, epsilon=0.01)

model.fit(x_train, y_train[:, 0])

y_train_pred = model.predict(x_train).reshape(-1, 1)
y_test_pred = model.predict(x_test).reshape(-1, 1)

y_train_pred = scaler.inverse_transform(y_train_pred)
y_test_pred = scaler.inverse_transform(y_test_pred)

y_train = scaler.inverse_transform(y_train)
y_test = scaler.inverse_transform(y_test)

train_timestamps = vibration[(vibration.index < test_start_dt) & (vibration.index >= train_start_dt)].index[
                   timesteps - 1:]
test_timestamps = vibration[(vibration.index < test_end_dt) & (vibration.index >= test_start_dt)].index[
                  timesteps - 1:]
iteration_dict["MAPE"] = round(mape(y_test_pred, y_test) * 100, 2)
iteration_dict["RMSE"] = round(rmse(y_test_pred, y_test) * 100, 2)
iteration_dict["TMAPE"] = round(mean_absolute_percentage_error(y_test, y_test_pred),2)
iteration_dict["TRMSE"] = round(mean_squared_error(y_test, y_test_pred, squared=False),2)

iteration_dict["Time Taken"] = round((time() - start) / 60, 2)
print(iteration_dict)

# plt.plot(train)

[[0.12393162]
 [0.13247863]
 [0.15811966]
 ...
 [0.06410256]
 [0.06837607]
 [0.06837607]]
{'MAPE': 4.94, 'RMSE': 4.0, 'TMAPE': 0.05, 'TRMSE': 0.04, 'Time Taken': 0.53}


In [21]:
print(y_test_pred)

[[0.70913592]
 [0.70771584]
 [0.72796648]
 ...
 [0.55423148]
 [0.57042975]
 [0.58244836]]
